In [597]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

df_students = pd.read_csv("students_complete.csv")
df_schools = pd.read_csv("schools_complete.csv")

In [598]:

df_schools.rename_axis({'name':'school'},axis=1,inplace=True)

#df_schools.rename(index=str, columns={"school": "name"})
df_students.head()

C:\Users\thiag\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  


,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [599]:
df_schools.head()
#schools_df.shape

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [545]:
df_school_district = df_schools[df_schools['type'] == 'District']
total_school_district= df_school_district['type'].count()
total_number_students_district = df_school_district['size'].sum()
total_district_budget = df_school_district['budget'].sum()
joined= pd.merge(df_students,df_school_district, how="left", on="school" )
#joined = joined[joined.columns[:8]]
joined = joined[["Student ID","name","gender","grade", "school","reading_score","math_score","type" ]]
filter_district= joined['type']=='District'
joined_2 = joined[filter_district]
joined_2.head()
average_reading_dis = joined_2['reading_score'].mean()
average_math_dis = joined_2['math_score'].mean()
Passing_math_count = joined_2.query('math_score >= 70').count()[0]
Passing_reading_count = joined_2[joined_2['reading_score'] >= 70].count()[1]
avg_passing_math_district = float(round(Passing_math_count / total_number_students_district,2))
avg_passing_reading_district = float(round(Passing_reading_count/ total_number_students_district,2))
overall_passing_district = (avg_passing_reading_district + avg_passing_math_district )/2

District_summary = pd.DataFrame({
       "Total_Schools": [total_school_district],
       "Total_Students": [total_number_students_district],
        "Total_Budget": [total_district_budget],
       "Average_Reading_score": [average_reading_dis],
        "Average_Math_score": [average_math_dis],
        "%Passing Math": [avg_passing_math_district],
        "%Passing Reading": [avg_passing_reading_district],
       "%Overall Passing Rate": [overall_passing_district]
})



In [546]:
District_summary = District_summary[["Total_Schools","Total_Students", "Total_Budget","Average_Reading_score","Average_Math_score","%Passing Math","%Passing Reading","%Overall Passing Rate" ]]
District_summary.head()

,Total_Schools,Total_Students,Total_Budget,Average_Reading_score,Average_Math_score,%Passing Math,%Passing Reading,%Overall Passing Rate
0,7,26976,17347923,80.962485,76.987026,0.67,0.81,0.74


In [608]:
df_school_summary = df_schools.copy()
df_school_summary = df_school_summary[df_school_summary.columns[1:5]]
df_school_summary['Budget per student'] = df_school_summary['budget']/df_school_summary['size']
avg_passing_math_reading_table = df_students.groupby(['school'])['reading_score','math_score'].mean().reset_index()
avg_passing_math_reading_table
df_school_summary = df_school_summary.merge(avg_passing_math_reading_table, on='school',how='outer')
summary_criteria_passing_reading  = df_students.query('reading_score >= 70')
summary_criteria_passing_math  = df_students.query('math_score >= 70')
passing_reading_count_summary =summary_criteria_passing_reading.groupby(['school'])['reading_score'].count().reset_index()
passing_reading_count_summary.rename_axis({'reading_score':'reading_count'},axis=1,inplace=True)
passing_math_count_summary = summary_criteria_passing_math.groupby(['school'])['math_score'].count().reset_index()
passing_math_count_summary.rename_axis({'math_score':'math_count'},axis=1,inplace=True)
passing_count = passing_math_count_summary.merge(passing_reading_count_summary,on='school',how='inner')


C:\Users\thiag\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  # Remove the CWD from sys.path while we load stuff.
C:\Users\thiag\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  if sys.path[0] == '':


In [609]:
df_school_summary =df_school_summary.merge(passing_count,on='school',how='outer')

In [610]:
df_school_summary['% Passing Math'] = (df_school_summary['math_count']/df_school_summary['size'])*100
df_school_summary['% Passing Reading'] = (df_school_summary['reading_count']/df_school_summary['size'])*100
df_school_summary= df_school_summary[["school","type", "budget","size","Budget per student","reading_score","math_score","% Passing Math","% Passing Reading" ]]
df_school_summary['% Overall Passing'] = (df_school_summary['% Passing Math'] + df_school_summary['% Passing Reading'])/2
df_school_summary.rename_axis({'reading_score':'Average Reading Score',
                               'math_score': 'Average Math Score'},axis= 1 , inplace= True)
df_school_summary

C:\Users\thiag\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  


,school,type,budget,size,Budget per student,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,1910635,2917,655.0,81.182722,76.629414,65.683922,81.316421,73.500171
1,Figueroa High School,District,1884411,2949,639.0,81.158020,76.711767,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1056600,1761,600.0,83.725724,83.359455,93.867121,95.854628,94.860875
3,Hernandez High School,District,3022020,4635,652.0,80.934412,77.289752,66.752967,80.862999,73.807983
4,Griffin High School,Charter,917500,1468,625.0,83.816757,83.351499,93.392371,97.138965,95.265668
5,Wilson High School,Charter,1319574,2283,578.0,83.989488,83.274201,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1081356,1858,582.0,83.975780,83.061895,94.133477,97.039828,95.586652
7,Bailey High School,District,3124928,4976,628.0,81.033963,77.048432,66.680064,81.933280,74.306672
8,Holden High School,Charter,248087,427,581.0,83.814988,83.803279,92.505855,96.252927,94.379391
9,Pena High School,Charter,585858,962,609.0,84.044699,83.839917,94.594595,95.945946,95.270270


In [611]:
df_school_summary_1 = df_school_summary.sort_values(['% Overall Passing'], ascending=[False]).head(5)
df_school_summary_1 # Top 5 schools

,school,type,budget,size,Budget per student,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,1081356,1858,582.0,83.975780,83.061895,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1043130,1635,638.0,83.848930,83.418349,93.272171,97.308869,95.290520
9,Pena High School,Charter,585858,962,609.0,84.044699,83.839917,94.594595,95.945946,95.270270
4,Griffin High School,Charter,917500,1468,625.0,83.816757,83.351499,93.392371,97.138965,95.265668
5,Wilson High School,Charter,1319574,2283,578.0,83.989488,83.274201,93.867718,96.539641,95.203679


In [612]:
df_school_summary_1 = df_school_summary.sort_values(['% Overall Passing'], ascending=[True]).head(5)
df_school_summary_1 # Bottom 5 schools

,school,type,budget,size,Budget per student,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
11,Rodriguez High School,District,2547363,3999,637.0,80.744686,76.842711,66.366592,80.220055,73.293323
1,Figueroa High School,District,1884411,2949,639.0,81.158020,76.711767,65.988471,80.739234,73.363852
0,Huang High School,District,1910635,2917,655.0,81.182722,76.629414,65.683922,81.316421,73.500171
12,Johnson High School,District,3094650,4761,650.0,80.966394,77.072464,66.057551,81.222432,73.639992
13,Ford High School,District,1763916,2739,644.0,80.746258,77.102592,68.309602,79.299014,73.804308


In [590]:
#* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
df_math_scores__grade =pd.pivot_table(df_students,values=['math_score'],aggfunc=np.mean,index=['school'],columns=['grade'])
#df_math_scores__grade = df_math_scores__grade.reindex_axis(labels=['9th',
                                                                  # '10th',
                                                                   #'11th',
                                                                   #'12th'],axis=1,level=1)
#Display Math Scores by Grade
#df_math_scores__grade.style.background_gradient(cmap=cm)
#df_math_scores__grade.style.background_gradient
df_math_scores__grade 



math_score                                 
grade                       10th       11th       12th        9th
school                                                           
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

In [613]:
#* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
df_math_scores__grade =pd.pivot_table(df_students,values=['reading_score'],aggfunc=np.mean,index=['school'],columns=['grade'])
df_math_scores__grade



reading_score                                 
grade                          10th       11th       12th        9th
school                                                              
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333

In [626]:

#Copy school summary dataframe to new dataframe titled scores by school spending#Copy sc 
scores_by_school_spending = df_school_summary.copy()

#assigned labels for bins
label_spending = np.array(['577.99 to 591.50', '591.51 to 628.00', '628.01 to 641.50', '641.51 to 655.00'])

#bin Per Student Budget column
scores_by_school_spending['Budget per student'] = pd.qcut(scores_by_school_spending['Budget per student'],4,labels=label_spending,precision=0)

#Display scores by school spending 
scores_by_school_spending.groupby(['Budget per student'])['Average Reading Score', 
                                                          'Average Math Score',
                                                          '% Passing Math',
                                                          '% Passing Reading',
                                                          '% Overall Passing'].mean()

,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
Budget per student,,,,,
577.99 to 591.50,83.933814,83.455399,93.460096,96.610877,95.035486
591.51 to 628.00,83.155286,81.899826,87.133538,92.718205,89.925871
628.01 to 641.50,81.917212,78.990942,75.209078,86.089386,80.649232
641.51 to 655.00,80.957446,77.023555,66.701010,80.675217,73.688113


In [621]:
#Copy school summary dataframe to new dataframe titled scores by school size
scores_by_school_size = df_school_summary.copy()

#label small, medium, and large for bins
label = np.array(['small','medium','large'])

#bin school size 
scores_by_school_size['size'] = pd.qcut(scores_by_school_size['size'],3,labels=label,precision=0)

#Use groupby to find the average scores for reading, math, % Passing Math, % Passing Reading, and % Overall Passing 
scores_by_school_size = scores_by_school_size.groupby(['size'])['Average Reading Score',
                                                                'Average Math Score', 
                                                                '% Passing Math',
                                                                '% Passing Reading',
                                                                '% Overall Passing'].mean()


scores_by_school_size

#reindex axis for dataframe 
#scores_by_school_size = scores_by_school_size.reindex_axis(labels=[2,1,0])
#scores_by_school_size.set_index(keys=['size'],inplace=True)

,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
size,,,,,
small,83.850220,83.554500,93.526422,96.500267,95.013345
medium,82.769850,80.750065,83.065610,90.161203,86.613407
large,80.967495,76.993025,66.369129,80.995600,73.682364


In [624]:

scores_by_school_type = df_school_summary.copy()



scores_by_school_type = scores_by_school_type.groupby(['type'])['Average Reading Score',
                                                                'Average Math Score',
                                                                '% Passing Math',
                                                                '% Passing Reading',
                                                                '% Overall Passing'].mean().reset_index()
#Set index to type
scores_by_school_type.set_index('type',inplace=True)
scores_by_school_type


,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.896421,83.473852,93.620830,96.586489,95.103660
District,80.966636,76.956733,66.548453,80.799062,73.673757
